In [1]:
from pymongo import MongoClient
import pandas as pd

def connect_mongodb(db_name, collection_name):
    # MongoDB 클라이언트 생성 (기본 포트 27017)
    client = MongoClient("mongodb://localhost:27017/")  # 변경: 본인의 MongoDB URI
    db = client[db_name]  # 데이터베이스 선택
    collection = db[collection_name]  # 컬렉션 선택
    return collection

# DataFrame을 MongoDB에 저장하는 함수
def save_to_mongodb_unique(df, collection):
    data_dict = df.to_dict("records")
    for record in data_dict:
        collection.update_one(
            {"content": record["content"]},  # content 필드를 기준으로 중복 확인
            {"$setOnInsert": record},        # 데이터가 없을 때만 삽입
            upsert=True                      # 조건에 맞는 데이터가 없으면 삽입
        )

In [2]:
# MongoDB에서 데이터를 읽어오는 함수
def read_from_mongodb(collection):
    data = list(collection.find())  # MongoDB에서 모든 데이터 가져오기
    return pd.DataFrame(data)       # DataFrame으로 변환

In [3]:
# DB에서 중복을 제거하는 함수
def remove_duplicates(collection, field_name):
    # 중복을 기준으로 조건 설정
    unique_items = collection.aggregate([
        {"$group": {
            "_id": f"${field_name}",
            "count": {"$sum": 1},
            "docs": {"$push": "$_id"}
        }},
        {"$match": {"count": {"$gt": 1}}}
    ])

    # 중복된 문서 삭제
    for item in unique_items:
        # 첫 번째 문서를 제외한 나머지 삭제
        docs_to_delete = item['docs'][1:]
        collection.delete_many({"_id": {"$in": docs_to_delete}})

In [24]:
# MongoDB 연결
# mydatabas 데이터베이스의 mycollection 컬렉션에 연결

collection = connect_mongodb(db_name="mydatabase", collection_name="mycollection")

Data saved to MongoDB successfully!


In [ ]:
remove_duplicates(collection, "content") # DB의 content 기준으로 중복 제거
save_to_mongodb_unique(results_df, collection) # 중복 제거된 데이터 저장
print("Data saved to MongoDB successfully!")

In [ ]:
from pymongo import MongoClient

# MongoDB 연결
client = MongoClient('mongodb://localhost:27017/')  # 연결 URL
db = client['mydatabase']  # 사용할 데이터베이스 이름
collection = db['mycollection']  # 사용할 컬렉션 이름

# 정리 함수 정의
def clean_text(text):
    """ 대괄호([])와 작은따옴표(') 제거하는 함수 """
    if isinstance(text, str):  # 문자열일 때만 처리
        return text.strip("[]").replace("'", "").strip()
    return text  # 문자열이 아니면 그대로 반환

# 데이터 정리 및 업데이트
for doc in collection.find():
    update_fields = {}  # 업데이트할 필드 저장용 딕셔너리
    
    # 정리할 필드 리스트
    fields_to_clean = ['address', 'comments', 'hashtags', 'restaurant name']
    
    for field in fields_to_clean:
        # 필드가 문서에 존재하면 정리 후 저장
        if field in doc:
            cleaned_text = clean_text(doc[field])
            if doc[field] != cleaned_text:  # 변경이 있을 때만 업데이트
                update_fields[field] = cleaned_text
    
    # 업데이트 실행
    if update_fields:
        collection.update_one({'_id': doc['_id']}, {'$set': update_fields})
        print(f"Updated: {update_fields}")

print("모든 필드 정리 완료!")


In [1]:
# DB 전처리 : 특정 필드가 '없음'인 문서 삭제
# 삭제 조건 : address 필드가 '없음'인 문서 삭제
result = collection.delete_many({"restaurant name": "없음"})
result = collection.delete_many({"address": "없음"})

# 삭제 결과 출력
print(f"삭제된 문서 수: {result.deleted_count}")

삭제된 문서 수: 0
